In [1]:
import math
import numpy as np
import pandas as pd
import os
import glob

from pykrx import stock
from pykrx import bond

import time
from time import sleep
from datetime import datetime
from datetime import timedelta

from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import psycopg2 as pg2
from sqlalchemy import create_engine

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


import warnings
warnings.filterwarnings('ignore')

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'


# BigQuery 클라이언트 객체 생성
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)


# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'    # 서비스 계정 생성한 bucket 이름 입력


now = datetime.now()
# now = now + timedelta(days=-2)
today_date1 = now.strftime('%Y%m%d')
today_date2 = now.strftime('%Y-%m-%d')
today_date_time_csv = now.strftime("%Y%m%d_%H%M")

# today_date1 = '2024010'
# today_date2 = '2023-01-07'

today_date1 = 'reset'

now = datetime.now()
now = now + timedelta(days=-(365 + 180))
set_date_1 = now.strftime('%Y%m%d')
query_date = now.strftime('%Y-%m-%d')

In [2]:
file_name = 'kor_index_ohlcv'
if not os.path.exists(f'data_crawler/cleaning/{file_name}'):
    os.makedirs(f'data_crawler/cleaning/{file_name}')

In [17]:
sql = f"""
SELECT 
  `date`,
  open, 
  high, 
  low, 
  close, 
  volume,
  trading_value,
  market_cap,
  `owenchoi-404302.finance_mlops.kor_index_ohlcv`.index_code,
  index_code_nm,
  market
FROM `owenchoi-404302.finance_mlops.kor_index_ohlcv` 
left join  `owenchoi-404302.finance_mlops.kor_index_list_df`
on `owenchoi-404302.finance_mlops.kor_index_ohlcv`.index_code = `owenchoi-404302.finance_mlops.kor_index_list_df`.index_code
where date > '{query_date}' and close != 0
order by date asc
"""

# 데이터 조회 쿼리 실행 결과
query_job = client.query(sql)

# 데이터프레임 변환
kor_index_ohlcv_2 = query_job.to_dataframe()


In [18]:
kor_index_ohlcv_2 = kor_index_ohlcv_2.fillna(0)
index_code_list = kor_index_ohlcv_2['index_code'].unique()


In [19]:
df_raw_total = pd.DataFrame()
df_raw_anal_total = pd.DataFrame()

for index_code_nm in index_code_list:
    df_raw = kor_index_ohlcv_2[kor_index_ohlcv_2['index_code'] == index_code_nm].reset_index(drop = True)

    ######################################################################
    # 보조지표
    ######################################################################

    # 이동평균선
    df_raw['MA5'] = df_raw['close'].rolling(window=5).mean()
    df_raw['MA20'] = df_raw['close'].rolling(window=20).mean()
    df_raw['MA60'] = df_raw['close'].rolling(window=60).mean()
    df_raw['MA120'] = df_raw['close'].rolling(window=120).mean()

    # 볼린저밴드
    std = df_raw['close'].rolling(20).std(ddof=0)

    df_raw['upper'] = df_raw['MA20'] + 2 * std
    df_raw['lower'] = df_raw['MA20'] - 2 * std

    # MACD
    # MACD
    macd = MACD(close=df_raw['close'],
                window_slow=26,
                window_fast=12,
                window_sign=9)


    df_raw['MACD_DIFF'] = macd.macd_diff()
    df_raw['MACD'] = macd.macd()
    df_raw['MACD_Signal'] = macd.macd_signal()

    # RSI
    df_raw['변화량'] = df_raw['close'] - df_raw['close'].shift(1)
    df_raw['변화량'] = df_raw['변화량'].astype('float64')
    df_raw['상승폭'] = np.where(df_raw['변화량']>=0, df_raw['변화량'], 0)
    df_raw['하락폭'] = np.where(df_raw['변화량'] <0, df_raw['변화량'].abs(), 0)

    # welles moving average
    df_raw['AU'] = df_raw['상승폭'].ewm(alpha=1/14, min_periods=14).mean()
    df_raw['AD'] = df_raw['하락폭'].ewm(alpha=1/14, min_periods=14).mean()
    df_raw['RSI'] = df_raw['AU'] / (df_raw['AU'] + df_raw['AD']) * 100

    df_raw['MA5-20'] = df_raw['MA5'] - df_raw['MA20']
    df_raw['MA20-60'] = df_raw['MA20'] - df_raw['MA60']
    df_raw['MA60-120'] = df_raw['MA60'] - df_raw['MA120']


    ######################################################################
    # 보조지표 분석
    ######################################################################
    df_raw_anal = df_raw[['date','index_code', 'index_code_nm','market', 'close']]

    # 골든크로스
    # 골든 크로스 5-20
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_5_20_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] > 0 and df_raw["MA5-20"].iloc[idx - 1] <= 0]

    # 데드 크로스 5-20
    # 양수에서 음수로 바뀌는 모든 인덱스 찾기
    idx_5_20_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] < 0 and df_raw["MA5-20"].iloc[idx - 1] >= 0]

    # 골든 크로스 20-60
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_20_60_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] > 0 and df_raw["MA20-60"].iloc[idx - 1] <= 0]

    # 골든 크로스 20-60
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_20_60_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] < 0 and df_raw["MA20-60"].iloc[idx - 1] >= 0]


    df_raw_anal.loc[:, '5_20_cross'] = '-'
    df_raw_anal.loc[idx_5_20_gold_cross,'5_20_cross'] = '골든크로스(매수)'
    df_raw_anal.loc[idx_5_20_dead_cross,'5_20_cross'] = '데드크로스(매도)'

    df_raw_anal.loc[:, '20_60_cross'] = '-'
    df_raw_anal.loc[idx_20_60_gold_cross,'20_60_cross'] = '골든크로스(매수)'
    df_raw_anal.loc[idx_20_60_dead_cross,'20_60_cross'] = '데드크로스(매도)'


    # 정배열 역배열
    ascending_sq  = (df_raw['MA5-20'] > 0) & \
    (df_raw['MA20-60'] > 0) & \
    (df_raw['MA60-120'] > 0)

    descending_sq  = (df_raw['MA5-20'] < 0) & \
    (df_raw['MA20-60'] < 0) & \
    (df_raw['MA60-120'] < 0)

    df_raw_anal.loc[:,'array'] = '-'
    df_raw_anal.loc[ascending_sq,'array'] = '정배열(매수)'
    df_raw_anal.loc[descending_sq,'array'] = '역배열(매도)'


    # 볼린저밴드
    df_raw['close'] = df_raw['close'].astype('float64')
    down_reg_sq = df_raw['upper'] - df_raw['close']
    top_reg_sq  = df_raw['lower'] - df_raw['close']

    down_reg = [idx for idx in range(1,len(df_raw)) if down_reg_sq[idx] > 0 and down_reg_sq[idx-1] <= 0]
    top_reg = [idx for idx in range(1,len(df_raw)) if top_reg_sq[idx] < 0 and top_reg_sq[idx-1] >= 0]

    df_raw_anal.loc[:,'Bollinger_band'] = '-'
    df_raw_anal.loc[down_reg,'Bollinger_band'] = '하향회귀(매도)'
    df_raw_anal.loc[top_reg,'Bollinger_band'] = '상향회귀(매수)'


    # MACD
    signal_down_cross = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] < 0 and df_raw['MACD_DIFF'][idx-1] >= 0]
    signal_top_corss = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] > 0 and df_raw['MACD_DIFF'][idx-1] <= 0]

    df_raw_anal.loc[:,'MACD'] = '-'
    df_raw_anal.loc[signal_down_cross,'MACD'] = '하향돌파(매도)'
    df_raw_anal.loc[signal_top_corss,'MACD'] = '상향돌파(매수)'

    # RSI
    down_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] > 70 and df_raw['RSI'][idx-1] <= 70]
    top_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] < 30 and df_raw['RSI'][idx-1] >= 30]


    df_raw_anal.loc[:,'RSI'] = '-'
    df_raw_anal.loc[down_reg,'RSI'] = 'RSI 상단 하향돌파(매도)'
    df_raw_anal.loc[top_reg,'RSI'] = 'RSI 하단 상향 돌파(매수)'


    df_raw_total = pd.concat([df_raw_total, df_raw])
    df_raw_anal_total = pd.concat([df_raw_anal_total, df_raw_anal])

    print(index_code_nm)


df_raw_total = df_raw_total.reset_index(drop = True)
df_raw_anal_total = df_raw_anal_total.reset_index(drop = True)

1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1024
1025
1026
1027
1028
1034
1035
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1167
1182
1224
1227
1232
1244
1894
2001
2002
2003
2004
2012
2015
2024
2026
2027
2029
2031
2037
2041
2042
2043
2056
2058
2062
2063
2065
2066
2067
2068
2070
2072
2074
2075
2077
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2181
2182
2183
2184
2203
2212
2213
2214
2215
2216
2217
2218
2189


In [20]:
df_raw_total.tail(3)

,date,open,high,low,close,volume,trading_value,market_cap,index_code,index_code_nm,...,MACD_Signal,변화량,상승폭,하락폭,AU,AD,RSI,MA5-20,MA20-60,MA60-120
35536,2024-02-14 00:00:00+00:00,1255.52,1301.85,1247.43,1295.41,21726743,1092836468000,76983657538070,2189,코스닥 글로벌,...,-34.901070,17.01,17.01,0.0,8.843212,9.112450,49.250270,-27.7815,-58.601333,32.595250
35537,2024-02-15 00:00:00+00:00,1309.81,1319.13,1293.79,1312.81,26583113,1041018061683,78305506222730,2189,코스닥 글로벌,...,-32.836640,17.40,17.40,0.0,9.454412,8.461561,52.770853,-4.3250,-62.961833,33.243667
35538,2024-02-16 00:00:00+00:00,1324.88,1327.22,1307.24,1313.28,16231267,807663817140,78334354362120,2189,코스닥 글로벌,...,-30.192457,0.47,0.47,0.0,8.812668,7.857164,52.865968,14.4080,-65.933667,34.100167


In [21]:
df_raw_anal_total.tail(3)

,date,index_code,index_code_nm,market,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
35536,2024-02-14 00:00:00+00:00,2189,코스닥 글로벌,KOSDAQ,1295.41,-,-,-,-,-,-
35537,2024-02-15 00:00:00+00:00,2189,코스닥 글로벌,KOSDAQ,1312.81,-,-,-,-,-,-
35538,2024-02-16 00:00:00+00:00,2189,코스닥 글로벌,KOSDAQ,1313.28,골든크로스(매수),-,-,-,-,-


In [12]:
now = datetime.now()
now = now + timedelta(days=-365)
set_date_1 = now.strftime('%Y%m%d')
set_date_2 = now.strftime('%Y-%m-%d')

df_raw_total_2 = df_raw_total[df_raw_total['date'] > set_date_2].reset_index(drop = True)
df_raw_anal_total_2 = df_raw_anal_total[df_raw_anal_total['date'] > set_date_2].reset_index(drop = True)



In [16]:
df_raw_anal_total_2

,date,index_code,index_code_nm,market,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
0,2023-02-17 00:00:00+00:00,1001,코스피,KOSPI,2451.21,-,-,정배열(매수),-,-,-
1,2023-02-20 00:00:00+00:00,1001,코스피,KOSPI,2455.12,데드크로스(매도),-,-,-,-,-
2,2023-02-21 00:00:00+00:00,1001,코스피,KOSPI,2458.96,-,-,-,-,-,-
3,2023-02-22 00:00:00+00:00,1001,코스피,KOSPI,2417.68,-,-,-,-,-,-
4,2023-02-23 00:00:00+00:00,1001,코스피,KOSPI,2439.09,-,-,-,상향회귀(매수),-,-
...,...,...,...,...,...,...,...,...,...,...,...
23760,2024-02-08 00:00:00+00:00,2189,코스닥 글로벌,KOSDAQ,1260.19,-,-,-,-,-,-
23761,2024-02-13 00:00:00+00:00,2189,코스닥 글로벌,KOSDAQ,1278.40,-,-,-,-,상향돌파(매수),-
23762,2024-02-14 00:00:00+00:00,2189,코스닥 글로벌,KOSDAQ,1295.41,-,-,-,-,-,-
23763,2024-02-15 00:00:00+00:00,2189,코스닥 글로벌,KOSDAQ,1312.81,-,-,-,-,-,-


In [41]:
## 매수 매도 카운트
max_date = max(df_raw_anal_total_2['date'])
buy_sell_count = df_raw_anal_total_2[df_raw_anal_total_2['date'] == max_date].reset_index(drop = True)

In [58]:
table_from_pandas = pa.Table.from_pandas(df_raw_total_2,preserve_index = False)
pq.write_table(table_from_pandas, f'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_cleaning.parquet')

table_from_pandas = pa.Table.from_pandas(df_raw_anal_total_2,preserve_index = False)
pq.write_table(table_from_pandas, f'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_anal_cleaning.parquet')


# Google Storage 적재
source_file_name = f'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_cleaning.parquet'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_cleaning.parquet'    # 업로드할 파일을 GCP에 저장할 때의 이름
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)


# Google Storage 적재
source_file_name = f'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_anal_cleaning.parquet'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_anal_cleaning.parquet'    # 업로드할 파일을 GCP에 저장할 때의 이름
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)